In [26]:
import os
import itertools
import shutil
import math
from configparser import ConfigParser
from datetime import datetime
from time import time, gmtime, strftime

import numpy as np
import pandas as pd
import tensorflow as tf
from utils import EarlyStoppingHook
from sklearn.metrics import roc_auc_score, accuracy_score

In [3]:
# configファイルの読み込み
config_filename = './config/NeuralFM_config.ini'

param_config = ConfigParser()
param_config.read(config_filename)


path
work_dir : /home/study/recommend_notebooks
model_dir : /home/study/recommend_notebooks/models/


train
work_dir : /home/study/recommend_notebooks
filename_pattern : /home/study/recommend_notebooks/data/MovieLens20M/classification/train20m.csv
batch_size : 128
max_steps : 1000000
num_epochs : 200
skip_header_lines : 1


eval
work_dir : /home/study/recommend_notebooks
filename_pattern : /home/study/recommend_notebooks/data/MovieLens20M/classification/eval20m.csv
batch_size : 128
max_steps : None
num_epochs : 1
skip_header_lines : 1


predict
work_dir : /home/study/recommend_notebooks
filename_pattern : /home/study/recommend_notebooks/data/MovieLens20M/classification/test20m.csv
batch_size : 128
max_steps : None
num_epochs : 1
skip_header_lines : 1


model
user_bucket_size : 8000
item_bucket_size : 12000
wide_feature_dim : 200000
user_embedding_dim : 258
item_embedding_dim : 258
dropout_prob : 0.3
early_stop : 2000




In [4]:
# データの入力
HEADER = ['user_id', 'item_id', 'rating']
HEADER_DEFAULTS = [['0'], ['0'], [0]]

FEATURE_NAMES = ['user_id', 'item_id']
CATEGORICAL_FEATURE_NAMES_WITH_BUCKET_SIZE = {
  'user_id': int(param_config['model']['user_bucket_size']),
  'item_id' : int(param_config['model']['item_bucket_size'])
  }

USED_FEATURE_NAMES = ['user_id', 'item_id', 'rating']

CATEGORICAL_FEATURE_NAMES =  list(CATEGORICAL_FEATURE_NAMES_WITH_BUCKET_SIZE.keys())
TARGET = 'rating'
TARGET_LABELS = [0,1]


In [5]:
user_embedding_dim = int(param_config['model']['user_embedding_dim'])
item_embedding_dim = int(param_config['model']['item_embedding_dim'])
    
# カラム情報取得
categorical_hash_user_raw = \
tf.feature_column.categorical_column_with_hash_bucket('user_id', CATEGORICAL_FEATURE_NAMES_WITH_BUCKET_SIZE['user_id'])
categorical_hash_item_raw = \
    tf.feature_column.categorical_column_with_hash_bucket('item_id', CATEGORICAL_FEATURE_NAMES_WITH_BUCKET_SIZE['item_id'])

categorical_hash_user = tf.feature_column.indicator_column(categorical_hash_user_raw)    
categorical_hash_item = tf.feature_column.indicator_column(categorical_hash_item_raw)
categorical_feature_linear = [categorical_hash_user, categorical_hash_item]
    
# 後半のEmbeddingパートを作成
categorical_feature_user_emb = tf.feature_column.embedding_column(
    categorical_column=categorical_hash_user_raw, dimension=user_embedding_dim)
categorical_feature_item_emb = tf.feature_column.embedding_column(
    categorical_column=categorical_hash_item_raw, dimension=item_embedding_dim)
categorical_feature_emb = [categorical_feature_user_emb, categorical_feature_item_emb]


In [6]:
params = {
    'categorical_feature_linear': categorical_feature_linear,
    'categorical_feature_emb': categorical_feature_emb,
    'hidden_units': [128, 64, 32],
    'dropout_prob': 0.3,
    'n_classes': 2
}

In [7]:
def NeuralFM_fn(features, labels, mode, params):
    input_features_linear = tf.feature_column.input_layer(features, params['categorical_feature_linear'])
    input_features_dnn = tf.feature_column.input_layer(features, params['categorical_feature_emb'])
    
    feature_sq_sum = tf.square(tf.reduce_sum(input_features_dnn, axis=1, keepdims=True))
    feature_sum_sq = tf.reduce_sum(tf.square(input_features_dnn), axis=1, keepdims=True)
    cross_term = 0.5 * tf.subtract(feature_sq_sum, feature_sum_sq)
    
    for hidden_unit in params['hidden_units']:
        layer = tf.layers.dense(cross_term, units=hidden_unit, activation=tf.nn.relu)
    layer = tf.nn.dropout(layer, rate=params['dropout_prob'])
    logits = tf.layers.dense(layer, params['n_classes'], activation=None)
    
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op')
    
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
    
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


In [8]:
def parse_input_config(config, phase):
    '''
    configをパースする関数（内部使用）
    configはいつもの設定ファイル
    phaseは{train, eval, predict}のいずれか
    batch_sizeやnum_epochsなどint型で入って欲しい変数がstrになってしまいintへの変換が必要となったためこの関数を用意した
    '''
    filename_pattern = config[phase]['filename_pattern']
    batch_size = int(config[phase]['batch_size'])
    try:
        num_epochs = int(config[phase]['num_epochs'])
    except:
        num_epochs = 1 # Noneにすると一生止まらない
    # これだけは正直共通にしておきたい
    skip_header_lines = int(config[phase]['skip_header_lines'])

    return filename_pattern, batch_size, num_epochs, skip_header_lines



In [9]:
def parse_csv_row(csv_row):
    '''
    csvをparseする関数
    途中でHEADERやHEADER_DEFAULTSを使用しているのでそれらを定義する関数か何かが必要
    （lambdaで使用しているため引数に加えることはできない）
    '''
    columns = tf.decode_csv(csv_row, record_defaults=HEADER_DEFAULTS)
    features = dict(zip(HEADER, columns))

    target = features.pop(TARGET)
    return features, target


In [10]:
def csv_input_fn(config, phase, mode=tf.estimator.ModeKeys.EVAL):
    '''
    ファイル名のパターンとか学習・評価時の設定は全てconfigに入れておいた方が管理しやすい
    （いつ・どういうタイミングで？）
    modeは直打ちで渡す方が楽そう（どこで？）
    '''
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    filename_pattern, batch_size, num_epochs, skip_header_lines = parse_input_config(config, phase)

    # ファイル名のパターンを元にデータの読み込み
    file_names = tf.matching_files(filename_pattern)
    dataset = tf.data.TextLineDataset(filenames=file_names)
    dataset = dataset.skip(skip_header_lines)

    # バッチサイズ分だけ切り出しgenerateする
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda csv_row: parse_csv_row(csv_row))
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()

    features, target = iterator.get_next()

    return features, target

In [11]:
def json_serving_input_fn():
    '''
    serving用のinput_fn
    同じくTARGETやUSED_FEATURE_NAMEを呼び出す関数がここにも必要
    '''
    receiver_tensor = {}
    for feature_name in USED_FEATURE_NAMES:
        dtype = tf.float32 if feature_name == TARGET else tf.string
        receiver_tensor[feature_name] = tf.placeholder(shape=[None], dtype=dtype)

    return tf.estimator.export.ServingInputReceiver(receiver_tensor, receiver_tensor)


In [12]:
train_input_fn = lambda: csv_input_fn(config=param_config, 
                                      phase='train', 
                                      mode=tf.estimator.ModeKeys.TRAIN)

eval_input_fn = lambda: csv_input_fn(config=param_config,
                                     phase='eval', 
                                     mode=tf.estimator.ModeKeys.EVAL)


In [13]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn,
                  max_steps=int(param_config['train']['max_steps']),
                  hooks=[EarlyStoppingHook(int(param_config['model']['early_stop']))]
                  )

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
                exporters=[tf.estimator.LatestExporter(name="estimate",  
                                                       serving_input_receiver_fn=json_serving_input_fn)],
                steps=None,
                throttle_secs = 15
                )


*** Early Stopping Hook: - Created
*** Early Stopping Hook:: Early Stopping Rounds: 2000



In [14]:
raw_execute_time = gmtime()
execute_time = strftime("%Y%m%d_%H%M%S", raw_execute_time )
model_dir = os.path.join(param_config['path']['model_dir'], execute_time)
print(model_dir)

/home/study/recommend_notebooks/models/20190504_155936


In [18]:
run_config = tf.estimator.RunConfig().replace(model_dir=model_dir,save_checkpoints_secs=300)


In [19]:
estimator = tf.estimator.Estimator(model_fn=NeuralFM_fn, 
                                  params=params, 
                                  config=run_config)

INFO:tensorflow:Using config: {'_model_dir': '/home/study/recommend_notebooks/models/20190504_155936', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6dd427bc18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/study/recommend_notebooks/models/20190504_155936/model.ckpt-764
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 764 into /home/study/recommend_notebooks/models/20190504_155936/model.ckpt.
Early Stopping Hook: No improvment! Counter: 38
************************

INFO:tensorflow:loss = 0.61712235, step = 765
Ea

Early Stopping Hook: No improvment! Counter: 37
************************

INFO:tensorflow:loss = 0.6303679, step = 865 (2.789 sec)
Early Stopping Hook: No improvment! Counter: 38
************************

Early Stopping Hook: No improvment! Counter: 39
************************

Early Stopping Hook: No improvment! Counter: 40
************************

Early Stopping Hook: No improvment! Counter: 41
************************

Early Stopping Hook: No improvment! Counter: 42
************************

Early Stopping Hook: No improvment! Counter: 43
************************

Early Stopping Hook: No improvment! Counter: 44
************************

Early Stopping Hook: No improvment! Counter: 45
************************

Early Stopping Hook: No improvment! Counter: 46
************************

Early Stopping Hook: No improvment! Counter: 47
************************

Early Stopping Hook: No improvment! Counter: 48
************************

Early Stopping Hook: No improvment! Counter: 49
*******

Early Stopping Hook: No improvment! Counter: 20
************************

Early Stopping Hook: No improvment! Counter: 21
************************

Early Stopping Hook: No improvment! Counter: 22
************************

Early Stopping Hook: No improvment! Counter: 23
************************

Early Stopping Hook: No improvment! Counter: 24
************************

Early Stopping Hook: No improvment! Counter: 25
************************

Early Stopping Hook: No improvment! Counter: 26
************************

Early Stopping Hook: No improvment! Counter: 27
************************

Early Stopping Hook: No improvment! Counter: 28
************************

Early Stopping Hook: No improvment! Counter: 29
************************

Early Stopping Hook: No improvment! Counter: 30
************************

Early Stopping Hook: No improvment! Counter: 31
************************

Early Stopping Hook: No improvment! Counter: 32
************************

************************

Early Stoppi

Early Stopping Hook: No improvment! Counter: 98
************************

Early Stopping Hook: No improvment! Counter: 99
************************

Early Stopping Hook: No improvment! Counter: 100
************************

Early Stopping Hook: No improvment! Counter: 101
************************

Early Stopping Hook: No improvment! Counter: 102
************************

Early Stopping Hook: No improvment! Counter: 103
************************

Early Stopping Hook: No improvment! Counter: 104
************************

Early Stopping Hook: No improvment! Counter: 105
************************

Early Stopping Hook: No improvment! Counter: 106
************************

Early Stopping Hook: No improvment! Counter: 107
************************

Early Stopping Hook: No improvment! Counter: 108
************************

Early Stopping Hook: No improvment! Counter: 109
************************

Early Stopping Hook: No improvment! Counter: 110
************************

Early Stopping Hook: No imp

Early Stopping Hook: No improvment! Counter: 17
************************

Early Stopping Hook: No improvment! Counter: 18
************************

Early Stopping Hook: No improvment! Counter: 19
************************

Early Stopping Hook: No improvment! Counter: 20
************************

Early Stopping Hook: No improvment! Counter: 21
************************

Early Stopping Hook: No improvment! Counter: 22
************************

Early Stopping Hook: No improvment! Counter: 23
************************

Early Stopping Hook: No improvment! Counter: 24
************************

Early Stopping Hook: No improvment! Counter: 25
************************

Early Stopping Hook: No improvment! Counter: 26
************************

Early Stopping Hook: No improvment! Counter: 27
************************

Early Stopping Hook: No improvment! Counter: 28
************************

Early Stopping Hook: No improvment! Counter: 29
************************

Early Stopping Hook: No improvment! Co

...

Early Stopping Hook: No improvment! Counter: 1609
************************

Early Stopping Hook: No improvment! Counter: 1610
************************

Early Stopping Hook: No improvment! Counter: 1611
************************

Early Stopping Hook: No improvment! Counter: 1612
************************

Early Stopping Hook: No improvment! Counter: 1613
************************

Early Stopping Hook: No improvment! Counter: 1614
************************

Early Stopping Hook: No improvment! Counter: 1615
************************

Early Stopping Hook: No improvment! Counter: 1616
************************

Early Stopping Hook: No improvment! Counter: 1617
************************

Early Stopping Hook: No improvment! Counter: 1618
************************

Early Stopping Hook: No improvment! Counter: 1619
************************

Early Stopping Hook: No improvment! Counter: 1620
************************

Early Stopping Hook: No improvment! Counter: 1621
************************

Early Stoppi

Early Stopping Hook: No improvment! Counter: 1716
************************

Early Stopping Hook: No improvment! Counter: 1717
************************

Early Stopping Hook: No improvment! Counter: 1718
************************

Early Stopping Hook: No improvment! Counter: 1719
************************

Early Stopping Hook: No improvment! Counter: 1720
************************

Early Stopping Hook: No improvment! Counter: 1721
************************

Early Stopping Hook: No improvment! Counter: 1722
************************

Early Stopping Hook: No improvment! Counter: 1723
************************

Early Stopping Hook: No improvment! Counter: 1724
************************

Early Stopping Hook: No improvment! Counter: 1725
************************

Early Stopping Hook: No improvment! Counter: 1726
************************

Early Stopping Hook: No improvment! Counter: 1727
************************

Early Stopping Hook: No improvment! Counter: 1728
************************

Early Stoppi

Early Stopping Hook: No improvment! Counter: 1825
************************

Early Stopping Hook: No improvment! Counter: 1826
************************

Early Stopping Hook: No improvment! Counter: 1827
************************

Early Stopping Hook: No improvment! Counter: 1828
************************

Early Stopping Hook: No improvment! Counter: 1829
************************

Early Stopping Hook: No improvment! Counter: 1830
************************

Early Stopping Hook: No improvment! Counter: 1831
************************

Early Stopping Hook: No improvment! Counter: 1832
************************

Early Stopping Hook: No improvment! Counter: 1833
************************

Early Stopping Hook: No improvment! Counter: 1834
************************

Early Stopping Hook: No improvment! Counter: 1835
************************

Early Stopping Hook: No improvment! Counter: 1836
************************

Early Stopping Hook: No improvment! Counter: 1837
************************

Early Stoppi

Early Stopping Hook: No improvment! Counter: 1932
************************

Early Stopping Hook: No improvment! Counter: 1933
************************

Early Stopping Hook: No improvment! Counter: 1934
************************

Early Stopping Hook: No improvment! Counter: 1935
************************

Early Stopping Hook: No improvment! Counter: 1936
************************

Early Stopping Hook: No improvment! Counter: 1937
************************

Early Stopping Hook: No improvment! Counter: 1938
************************

Early Stopping Hook: No improvment! Counter: 1939
************************

Early Stopping Hook: No improvment! Counter: 1940
************************

Early Stopping Hook: No improvment! Counter: 1941
************************

Early Stopping Hook: No improvment! Counter: 1942
************************

Early Stopping Hook: No improvment! Counter: 1943
************************

Early Stopping Hook: No improvment! Counter: 1944
************************

Early Stoppi

({'accuracy': 0.74311477, 'loss': 0.7285554, 'global_step': 8291},
 [b'/home/study/recommend_notebooks/models/20190504_155936/export/estimate/1556985935'])

## 予測&評価

In [21]:
test_data = pd.read_csv(param_config['predict']['filename_pattern'])
test_size = len(test_data)


In [22]:
predict_input_fn = lambda: csv_input_fn(config=param_config, 
                                      phase='predict', 
                                      mode=tf.estimator.ModeKeys.PREDICT)


In [23]:
predictions = estimator.predict(input_fn=predict_input_fn)

In [24]:
values = list(map(lambda item: item["class_ids"][0],list(itertools.islice(predictions, test_size))))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/study/recommend_notebooks/models/20190504_155936/model.ckpt-8291
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [27]:
test_value = np.array(test_data.iloc[:,2])
pred_value = np.array(values)
pred_value_binary = np.round(pred_value)

auc = roc_auc_score(test_value, pred_value)
accuracy = accuracy_score(test_value, pred_value_binary)
print('AUC: {:.4f}\nAccuracy: {:.4f}'.format(auc, accuracy))


AUC: 0.7054
Accuracy: 0.7054


In [ ]:
result = ','.join([config_filename,str(auc_score), str(accuracy_score),execute_time]) + '\n'

with open('../log/result.csv', 'a') as f:
    f.write(result)


In [ ]:
result_df = pd.read_csv('../log/result.csv')
result_df.tail()
